<a href="https://colab.research.google.com/github/manojec054/Assignment/blob/master/Copy_of_Credit_Card_Fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Credit Card Fraud

Data Has been sourced from Kaggle

Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/downloads/creditcardfraud.zip


The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Our task is to make a simple DL classifer to correctly classify frauds.

In [1]:
# Starting with useful imports
import csv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [3]:
%ls

MyDrive/


In [4]:
fname = 'My Drive/Data/UCI_Credit_Card.csv'
all_features = []
all_targets = []
with open(fname) as f:
  for i, line in enumerate(f):
    if i == 0:
      print('HEADER:', line.strip())
      continue  # Skip header
    fields = line.strip().split(',')
    all_features.append([float(v.replace('"', '')) for v in fields[:-1]])
    all_targets.append([int(fields[-1].replace('"', ''))])
    if i == 1:
      print('EXAMPLE FEATURES:', all_features[-1])
    
features = np.array(all_features, dtype='float32')
targets = np.array(all_targets, dtype='uint8')
print('features.shape:', features.shape)
print('targets.shape:', targets.shape)

HEADER: "ID","LIMIT_BAL","SEX","EDUCATION","MARRIAGE","AGE","PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6","default.payment.next.month"
EXAMPLE FEATURES: [1.0, 20000.0, 2.0, 2.0, 1.0, 24.0, 2.0, 2.0, -1.0, -1.0, -2.0, -2.0, 3913.0, 3102.0, 689.0, 0.0, 0.0, 0.0, 0.0, 689.0, 0.0, 0.0, 0.0, 0.0]
features.shape: (30000, 24)
targets.shape: (30000, 1)


In [5]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('My Drive/Data/UCI_Credit_Card.csv')
df.rename(columns={'default.payment.next.month':'target', 'PAY_0':'PAY_1'}, inplace=True)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,target
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [6]:
display(df.EDUCATION.value_counts())
fil = (df.EDUCATION == 5) | (df.EDUCATION == 6) | (df.EDUCATION == 0)
df.loc[fil, 'EDUCATION'] = 4
df.EDUCATION.value_counts()

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64

2    14030
1    10585
3     4917
4      468
Name: EDUCATION, dtype: int64

In [7]:
df.loc[df.MARRIAGE == 0, 'MARRIAGE'] = 3
df.MARRIAGE.value_counts()

2    15964
1    13659
3      377
Name: MARRIAGE, dtype: int64

In [8]:
df.target.value_counts()

0    23364
1     6636
Name: target, dtype: int64

In [9]:
df['AgeBin'] = 0 #creates a column of 0
df.loc[((df['AGE'] > 20) & (df['AGE'] < 30)) , 'AgeBin'] = 1
df.loc[((df['AGE'] >= 30) & (df['AGE'] < 40)) , 'AgeBin'] = 2
df.loc[((df['AGE'] >= 40) & (df['AGE'] < 50)) , 'AgeBin'] = 3
df.loc[((df['AGE'] >= 50) & (df['AGE'] < 60)) , 'AgeBin'] = 4
df.loc[((df['AGE'] >= 60) & (df['AGE'] < 70)) , 'AgeBin'] = 5
df.loc[((df['AGE'] >= 70) & (df['AGE'] < 81)) , 'AgeBin'] = 6
df.loc[(df['AgeBin'] == 6) , 'AgeBin'] = 5

In [10]:
df['SE_MA_2'] = 0
df.loc[((df.SEX == 1) & (df.MARRIAGE == 1)) , 'SE_MA_2'] = 1 # Married male 
df.loc[((df.SEX == 1) & (df.MARRIAGE == 2)) , 'SE_MA_2'] = 2 # Single male 
df.loc[((df.SEX == 1) & (df.MARRIAGE == 3)) , 'SE_MA_2'] = 3 
df.loc[((df.SEX == 2) & (df.MARRIAGE == 1)) , 'SE_MA_2'] = 4 
df.loc[((df.SEX == 2) & (df.MARRIAGE == 2)) , 'SE_MA_2'] = 5 
df.loc[((df.SEX == 2) & (df.MARRIAGE == 3)) , 'SE_MA_2'] = 6 

In [11]:
df['SE_AG'] = 0
df.loc[((df.SEX == 1) & (df.AgeBin == 1)) , 'SE_AG'] = 1 #man in 20's
df.loc[((df.SEX == 1) & (df.AgeBin == 2)) , 'SE_AG'] = 2 #man in 30's
df.loc[((df.SEX == 1) & (df.AgeBin == 3)) , 'SE_AG'] = 3 #man in 40's
df.loc[((df.SEX == 1) & (df.AgeBin == 4)) , 'SE_AG'] = 4 #man in 50's
df.loc[((df.SEX == 1) & (df.AgeBin == 5)) , 'SE_AG'] = 5 #man in 60's and above
df.loc[((df.SEX == 2) & (df.AgeBin == 1)) , 'SE_AG'] = 6 #woman in 20's
df.loc[((df.SEX == 2) & (df.AgeBin == 2)) , 'SE_AG'] = 7 #woman in 30's
df.loc[((df.SEX == 2) & (df.AgeBin == 3)) , 'SE_AG'] = 8 #woman in 40's
df.loc[((df.SEX == 2) & (df.AgeBin == 4)) , 'SE_AG'] = 9 #woman in 50's
df.loc[((df.SEX == 2) & (df.AgeBin == 5)) , 'SE_AG'] = 10 #woman in 60's and above

In [12]:
df['Proximity_6'] = (df.LIMIT_BAL - df.BILL_AMT6) / df.LIMIT_BAL
df['Proximity_5'] = (df.LIMIT_BAL - df.BILL_AMT5) / df.LIMIT_BAL
df['Proximity_4'] = (df.LIMIT_BAL - df.BILL_AMT4) / df.LIMIT_BAL
df['Proximity_3'] = (df.LIMIT_BAL - df.BILL_AMT3) / df.LIMIT_BAL
df['Proximity_2'] = (df.LIMIT_BAL - df.BILL_AMT2) / df.LIMIT_BAL
df['Proximity_1'] = (df.LIMIT_BAL - df.BILL_AMT1) / df.LIMIT_BAL

In [13]:
from sklearn.utils import resample
# Upsample minority class
upsampled = resample(df[df.target == 1], 
                                 replace=True,     # sample with replacement
                                 n_samples=23364,    # to match majority class
                                 random_state=587) # reproducible results
# Combine majority class with upsampled minority class
upsampled_df = pd.concat([df[df.target == 0], upsampled])
# Display new class counts
display(upsampled_df.target.value_counts())
upsampled_df.shape

1    23364
0    23364
Name: target, dtype: int64

(46728, 34)

In [14]:
from sklearn.utils import resample
# Downsample minority class
down_sampled = resample(df[df.target == 0], 
                                 replace=True,     # sample with replacement
                                 n_samples=6636,    # to match majority class
                                 random_state=587) # reproducible results
# Combine majority class with upsampled minority class
down_df = pd.concat([df[df.target == 1], down_sampled])
# Display new class counts
display(down_df.target.value_counts())
down_df.shape

1    6636
0    6636
Name: target, dtype: int64

(13272, 34)

In [15]:

df_final = upsampled_df
#df_final = down_df

In [16]:
df_final.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,target,AgeBin,SE_MA_2,SE_AG,Proximity_6,Proximity_5,Proximity_4,Proximity_3,Proximity_2,Proximity_1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,2,5,7,0.827233,0.833911,0.840767,0.849344,0.844144,0.675122
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,2,4,7,0.409060,0.420820,0.433720,0.014180,0.035340,0.060200
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,4,1,4,0.617380,0.617080,0.581200,0.283300,0.886600,0.827660
5,6,50000.0,1,1,2,37,0,0,0,0,0,0,64400.0,57069.0,57608.0,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0,2,2,2,0.599520,0.607620,0.612120,-0.152160,-0.141380,-0.288000
6,7,500000.0,1,1,2,29,0,0,0,0,0,0,367965.0,412023.0,445007.0,542653.0,483003.0,473944.0,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,0,1,2,1,0.052112,0.033994,-0.085306,0.109986,0.175954,0.264070


In [17]:
X = df_final.drop(['target','ID'], axis=1) 
y = df_final.target

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [19]:
train_features = X_train.to_numpy()
train_targets = y_train.to_numpy().reshape(-1, 1)
test_features = X_test.to_numpy()
test_targets = y_test.to_numpy().reshape(-1, 1)

In [20]:
num_test_samples = int(len(features) * 0.2)
#train_features = features[:-num_test_samples]
#train_targets = targets[:-num_test_samples]
#test_features = features[-num_test_samples:]
#test_targets = targets[-num_test_samples:]
print('Number of training samples:', len(train_features))
print('Number of test samples:', len(test_features))


Number of training samples: 37382
Number of test samples: 9346


In [21]:
counts = np.bincount( train_targets[:, 0])
print('Number of positive samples in training data: {} ({:.2f}% of total)'.format(counts[1], 100 * float(counts[1]) / len(train_targets)))

Number of positive samples in training data: 18715 (50.06% of total)


In [22]:
counts = np.bincount(test_targets[:, 0])
print('Number of positive samples in testing data: {} ({:.2f}% of total)'.format(counts[1], 100 * float(counts[1]) / len(test_targets)))

Number of positive samples in testing data: 4649 (49.74% of total)


In [23]:
mean = np.mean(train_features, axis=0)
train_features -= mean
test_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
test_features /= std

In [24]:
train_features.shape

(37382, 32)

In [25]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import  Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(1024, activation=tf.nn.tanh))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation=tf.nn.tanh))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='softmax'))

#model.compile(optimizer='adam', loss='categorical_crossentropy')
# your code here

In [26]:
#model.fit(train_features, train_targets, epochs=100, batch_size=100)

In [27]:
metrics = [keras.metrics.FalseNegatives(name='fn'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.TruePositives(name='tp'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.Accuracy(name='accuracy')]

model.compile(optimizer=keras.optimizers.Adam(1e-2),
              loss='binary_crossentropy',
              metrics=metrics)

#callbacks = [keras.callbacks.ModelCheckpoint('fraud_model_at_epoch_{epoch}.h5')]
class_weight = {0: 1, 1: 200}

model.fit(train_features, train_targets,
          batch_size=4096, epochs=100, verbose=1, validation_data=(test_features, test_targets), class_weight=class_weight)

Epoch 1/100
10/10 [==============================] - 3s 103ms/step - loss: 23.2571 - fn: 0.0000e+00 - fp: 11747.3636 - tn: 0.0000e+00 - tp: 11805.7273 - precision: 0.5012 - recall: 1.0000 - accuracy: 0.5012 - val_loss: 4.7408 - val_fn: 0.0000e+00 - val_fp: 4697.0000 - val_tn: 0.0000e+00 - val_tp: 4649.0000 - val_precision: 0.4974 - val_recall: 1.0000 - val_accuracy: 0.4974
Epoch 2/100
10/10 [==============================] - 0s 16ms/step - loss: 3.9199 - fn: 0.0000e+00 - fp: 11732.0000 - tn: 0.0000e+00 - tp: 11821.0909 - precision: 0.5026 - recall: 1.0000 - accuracy: 0.5026 - val_loss: 3.5042 - val_fn: 0.0000e+00 - val_fp: 4697.0000 - val_tn: 0.0000e+00 - val_tp: 4649.0000 - val_precision: 0.4974 - val_recall: 1.0000 - val_accuracy: 0.4974
Epoch 3/100
10/10 [==============================] - 0s 14ms/step - loss: 3.2839 - fn: 0.0000e+00 - fp: 11745.2727 - tn: 0.0000e+00 - tp: 11807.8182 - precision: 0.5025 - recall: 1.0000 - accuracy: 0.5025 - val_loss: 3.0376 - val_fn: 0.0000e+00 - val

In [28]:
np.set_printoptions(suppress=True)
res = model.predict(test_features).reshape(1, -1)
np.array(np.unique(res, return_counts=True)).T

array([[   1., 9346.]])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              33792     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 722,945
Trainable params: 722,945
Non-trainable params: 0
_________________________________________________________________


In [30]:
# homework
# Solve the clasification problem. 
# handle data imbalance issue
# Design DL network to classify
# Print loss, F1 score and ROC
# Visualize loss in training and test data using matplotlib
# Send the link of colab to amritansh.48@gmail.com